In [4]:
using CurveFit
using FFTW
using Distributed
using PyPlot
pygui(true)
using PyCall
using Distributions
using TimeseriesSurrogates
pd=pyimport("pandas")

PyObject <module 'pandas' from '/home/rabindev/anaconda3/lib/python3.7/site-packages/pandas/__init__.py'>

In [ ]:
a = 0.5
typeof(a) # Float64 by default
b = convert(Float32,a)
typeof(b) # Now it is a Float3

In [2]:
# plt=pyimport("matplotlib.pyplot")
rc("axes", linewidth=3)
rc("font", family="serif")
size=36
rc("xtick", labelsize=size)
rc("ytick", labelsize=size)
rc("axes", labelsize=size)

# All required function

In [5]:
function FFT(X::AbstractArray{T},fs::Real) where T<:Real
    N=length(X)
    time=np.arange(length(X))./fs
    Fourier_coefficient=np.abs(fft(X))
    First_half_coefficient=Fourier_coefficient[1:trunc(Int,N/2)]
    Frequency=np.arange(trunc(Int,N/2))*fs/N
    return Frequency,First_half_coefficient
    
end

@everywhere function Invariant_density(X::AbstractArray{T},n::Real) where T<:Real
xstep=np.linspace(np.min(X),np.max(X),n)
xx=zeros(Float64,n-1)
Pxx=zeros(Float64,n-1)
  @distributed (+) for i=1:(length(xstep)-1)
    count=0
     for j=1:length(X)
        if xstep[i]<X[j]<xstep[i+1]
            count=count+1
            end
        if count==0
            nothing
            end
    end
    xx[i]=xstep[i]
    Pxx[i]=count/length(X)
end
return xx,Pxx
end

function q_sqrt(X::Array{Float64,1},q::Real)
         val=np.sum(X.^(q/2))
              
    return (val/length(X))^(1/q)
end
function q_sqrt_zero(X::Array{Float64,1})
    
        val=np.sum(np.log(X))
    return np.exp(val/(2*length(X)))
end


@everywhere function DFA(X::AbstractArray{T},q::Real,s::Int64,m::Int64) where T<:Real
    Y=np.cumsum(X.-np.mean(X))
    T1=float(np.arange(length(X)))
    N=length(Y)
    N_steps=trunc(Int,(N/s))
    F_aux=zeros(Float64,2*N_steps)
    
@distributed (+) for n=1:N_steps
    Y_aux1=Y[1+(n-1)*s:n*s]
    Y_aux2=Y[1+N-n*s:N-(n-1)*s]

    T_aux1=T1[1+(n-1)*s:n*s]
    T_aux2=T1[1+N-n*s:N-(n-1)*s]
    fit_out1=curve_fit(Poly,T_aux1,Y_aux1,m)
    fit_out2=curve_fit(Poly,T_aux2,Y_aux2,m)
    yv1=fit_out1.(T_aux1)
    yv2=fit_out2.(T_aux2)
    F_aux[n]=np.sum((Y_aux1.-yv1).^2)/length(yv1)
    F_aux[N_steps+n]=np.sum((Y_aux2.-yv2).^2)/length(yv2)   
    end
    return q,s,F_aux
end

function Conversion(Y)
    X=zeros(length(Y))
    @inbounds begin
        @simd for i=1:length(Y)
        X[i]=convert(Float64,Y[i])
    end
    end
    return X
end

function Finding_song_segment(X,Time,ti,tf)  
    T4=[]
    Y4=[]
    for i=1:length(X)
        if ti<=Time[i]<=tf
            append!(Y4,X[i])
            append!(T4,Time[i])
        end
    end
    return T4,Y4
end

function MFDFA(ym::AbstractArray{T},Q_mul::AbstractArray{T},SL::Array{Int64,1},order::Int64) where T<:Real
q=np.zeros(length(Q_mul))
Hq=np.zeros(length(Q_mul))
FQS=zeros(length(SL),length(Q_mul))
for j=1:length(Q_mul)
    if  Q_mul[j]==0.0
        q_aux=Q_mul[j]
        q[j]=q_aux
            
        SS=zeros(Float64,length(SL))
        Fq=zeros(Float64,length(SL)) 
        
        for i=1:length(SL)
            SS[i]=SL[i]
            dfa_out1=DFA(ym,q_aux,SL[i],order)
            Fq[i]=q_sqrt_zero(dfa_out1[3])
            FQS[i,j]=q_sqrt_zero(dfa_out1[3])
        end 
    FIT1=curve_fit(Poly,np.log10(SS),np.log10(Fq),1)
    coefficient1=FIT1.a
    Hq[j]=coefficient1[2]
    end
    if Q_mul[j]!=0
        q_aux=Q_mul[j]
        q[j]=q_aux
        SS=zeros(length(SL))
        Fq=zeros(length(SL)) 
        for i=1:length(SL)
            SS[i]=SL[i]
            dfa_out2=DFA(ym,q_aux,SL[i],order)
            FQ=q_sqrt(dfa_out2[3],q_aux)
            Fq[i]=FQ
            FQS[i,j]= FQ
            end 
     FIT2=curve_fit(Poly,np.log10(SS),np.log10(Fq),1)
     coefficient2=FIT2.a
     Hq[j]=coefficient2[2] 
        end 
 end
    return Q_mul,Hq,FQS
end        

MFDFA (generic function with 1 method)

In [6]:
function Shannon(X::AbstractArray{T},n::Int64) where T<: Real
    prob=Invariant_density(X,n)
    SH=0.0
    P=prob[2]
    for i=1:length(P)
        if P[i]==0.0
            term=0.0
            SH=SH+term
            else 
        term=P[i]*np.log10(P[i])
        SH=SH+term 
        end
    end
    return -SH
end

Shannon (generic function with 1 method)

In [7]:
function ZCR(X::AbstractArray{T}) where T<: Real
    Count=0
    N=length(X)
    for i=2:N-1
        if X[i+1]*X[i]<0  
            Count=Count+1
            elseif X[i-1]*X[i]<0
            Count=Count+1
        end
    return Count/(2*N)    
        
    end
end

ZCR (generic function with 1 method)

# White noise and Brown noise

In [8]:
using PyCall
np=pyimport("numpy")
# Wn=np.random.normal(0,1,500000);
# Bn=np.cumsum(Wn);

In [ ]:
y = np.random.normal(size=10);

In [ ]:
y, fs = wavread("/../../home/rabindev/All_sound_files/sound_files/PDH_283_cut1/PDH_283_cut1.wav")
ti=collect(1:length(y))./fs
plot(ti,y)

In [ ]:
y4=Finding_song_segment(y,time,6.34,10.62);
v=Conversion(y4[2]);

# Reading the wav file

In [38]:
using CSV
Q=[2.0]
m=1
S=np.arange(10,100,5)


HQ_all=[]
for i=1:10
data = CSV.read("/../../home/rabindev/Documents/Vocal_developement_in_Juvenile_Adult/Files_and_figures/canary_cortesindo/seg_4_54_to_6_25/Subsegments/syllable$i.csv")
# y, fs = wavread("/../../home/rabindev/All_sound_files/sound_files/canary_revisit_old/cortesonido_denoised.wav");

# # time=collect(1:length(y))./fs
# y4=Finding_song_segment(y,time,0,time[end]);
# v=Conversion(y4[2]);
yn=data.voltage
tn=data.time    
hil=pyimport("scipy.signal")
input_data=yn
amp_env=np.abs(hil.hilbert(input_data));
ym=amp_env
# Q=np.arange(-5,5.26,0.25)
# Q=[-5.0,-2.0,2.0,5.0]

FQS_result=@time MFDFA(ym,Q,S,m)
FQS==FQS_result[3];
    
append!(HQ_all,FQS_result[2])
end

  0.012594 seconds (13.28 k allocations: 1.956 MiB)
  0.027119 seconds (41.27 k allocations: 6.729 MiB)
  0.042105 seconds (47.30 k allocations: 7.266 MiB, 14.63% gc time)
  0.032814 seconds (52.33 k allocations: 8.637 MiB)
  0.044995 seconds (54.49 k allocations: 8.365 MiB, 13.33% gc time)
  0.036289 seconds (52.33 k allocations: 8.637 MiB)
  0.033086 seconds (51.47 k allocations: 8.480 MiB)
  0.052314 seconds (61.67 k allocations: 9.490 MiB, 11.32% gc time)
  0.035480 seconds (57.56 k allocations: 9.534 MiB)
  0.052293 seconds (65.63 k allocations: 10.113 MiB, 15.93% gc time)


In [39]:
HQ_all,np.mean(HQ_all),np.median(HQ_all)

(Any[1.41166, 1.83168, 1.88559, 1.85965, 1.93064, 1.87956, 1.91522, 1.93854, 1.83149, 1.90857], 1.839261012304752, 1.882573097517521)

# Finding amplitude envelope

In [ ]:
using WAV
y, fs = wavread("/../../home/rabindev/Documents/Vocal_developement_in_Juvenile_Adult/Files_and_figures/P2007_12_07_cut1/P2007_12_07_cut1_denoised.wav");
ti=collect(1:length(y))./fs
y4=Finding_song_segment(y,ti,0.825,5.1);
v=Conversion(y4[2])

hil=pyimport("scipy.signal")
input_data=v
amp_env=np.abs(hil.hilbert(input_data));
plot(input_data)
plot(amp_env)

In [ ]:
inv1=@time Invariant_density(amp_env,50)
plot(inv1[1],inv1[2],"ro-")

# Creating surrogate data

In [48]:
# z=8
using CSV
data = CSV.read("/../../home/rabindev/Documents/Vocal_developement_in_Juvenile_Adult/juvenile.csv")
# y1=data.voltage
# t1=data.time
# input_data=y1
# amp_env=np.abs(hil.hilbert(input_data));
# plot(input_data)
# plot(amp_env,color="C1")
# # shuf=randomshuffle(amp_env);  ## Uncorrelated surrogate data which has same probability distribution of the original data
# # surr=iaaft(amp_env);

In [49]:
ff=np.array(data.Fund);

In [40]:
length(ff)

1681

# MFDFA computation

In [50]:
ym=ff
# Q=np.arange(-5,5.26,0.25)
# Q=[-5.0,-2.0,2.0,5.0]
Q=[2.0]
m=1
S=np.arange(10,100,10)
FQS_result=@time MFDFA(ym,Q,S,m)
FQS=FQS_result[3];

  0.070829 seconds (35.73 k allocations: 5.541 MiB)


# Plot log F(s) vs log s

In [51]:
for k=1:length(Q)
    q_max=Q[k]
plot(np.log10(S),np.log10(FQS[:,k]),"o",color="C$(k-1)",markersize=15,linewidth=4,label="q=$q_max")
fit_aux=curve_fit(Poly,np.log10(S),np.log10(FQS[:,k]),1)
plot(np.log10(S),fit_aux.(np.log10(S)),color="k",linewidth=5)
slope=fit_aux.a
println(slope[2])    
end
legend(fontsize=25)
xlabel(L"log_{10}(s)")
ylabel(L"log_{10}F_{q}(s)")
# ylim(-2.8,1.5)
show()

1.2967637626405901


In [ ]:
Q_mul=FQS_result[1]
Hq=FQS_result[2]
hq_data=np.column_stack((Q_mul,Hq))
hq_data_frame=pd.DataFrame(hq_data,columns=["Q","Hq"])
hq_data_frame.to_csv("/../../home/rabindev/Bird_song_analysis_result/Juvenile_canary/PDH_283_cut1/seg_6_34_to_10_62/hq_q_s3_s_250_1000_order_2_amplitude_envelope.csv")
plot(Q_mul,Hq,"bo-",markersize=20,linewidth=4)
ylabel(L"$H_{q}$")
xlabel("q")
# text(-4,1.5,r'$\Delta H_{q} =%s$'%round((np.max(Hq)-np.min(Hq)),2))
# text(2.2,2.1," s_max=%s"%max(SL))
# text(2.2,2.3," s_min=%s"%min(SL))
# axhline(y=0,color='k',linewidth=3.5)
ylim(0,2.5)


# Hq vs q plot for original data and surrogate data

In [ ]:
LABEL=["Original data","Shuffle","Surrogate"]
for i=1:3
    hq_data_read=pd.read_csv("/../../home/rabindev/Bird_song_analysis_result/Juvenile_canary/PDH_283_cut1/seg_6_34_to_10_62/hq_q_s$(i)_s_250_1000_order_2_amplitude_envelope.csv")
    Q_read=np.array(hq_data_read.Q)
    Hq_read=np.array(hq_data_read.Hq)
    plot(Q_read,Hq_read,"o-",color="C$(i-1)",markersize=20,linewidth=4,label=LABEL[i])
    
    ylabel(L"$H_{q}$")
    xlabel("q")
    ylim(0,3.5)
legend(fontsize=30)    
end

In [ ]:
LABEL[1]

# Finding power spectrum

In [ ]:
using WAV
y, fs = wavread("/../../home/rabindev/Documents/Vocal_developement_in_Juvenile_Adult/Files_and_figures/canary_cortesindo/seg_11_12_to_12_27/Subsegment_using_praat/syllable4.wav");

y2=y./np.max(np.abs(y))
ti=collect(1:length(y))./fs
y4=Finding_song_segment(y,ti,ti[1],ti[end]);
v=Conversion(y4[2]);

out_ps=FFT(v,fs)
subplot(211)
plot(ti,v,color="C0")
xlabel("time (s)")
ylabel("Voltage (mV)")
subplot(212)
plot(out_ps[1],20*(np.log10(out_ps[2])),"C2.-")
xlabel("f(Hz)")
ylabel(L"S(f) (mV)$^2$")
xlim(0,10000)
